In [93]:
import os
import glob
import shutil

import youtube_dl

from pathlib import Path
import shutil
import glob


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS

import webvtt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from flask import Flask, request, render_template, jsonify
#os.chdir("/Users/shradhitsubudhi/Documents/youtube/downloads")




In [ ]:
app = Flask(__name__, static_url_path="")

@app.route('/')
def index():
    """Return the main page."""
    return render_template('theme.html')



app = Flask(__name__)

@app.route('/')
def index():
    return '<h1> Youtube Sentimental Analysis!</h1>'\

@app.route("/form_data", methods=["GET", "POST"])
def form_data():

    if request.method =="POST":
        link = request.form.get('link')

    scraping(link)

    return link





In [106]:
def scraping_download(link):
    
    source = os.getcwd()
    dest = os.getcwd() + '/down/'
    
    ydl_opts = {'writesubtitles': True,
                'writeautomaticsub': True,
                'writeinfojson': True,
                'format': 'bestaudio/best',
                'keepvideo': False,
                'postprocessors': [{'key': 'FFmpegExtractAudio',
                                    'preferredcodec': 'wav',
                                    'preferredquality': '192'}],
                'postprocessor_args': ['-ar', '16000']}


    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        meta = ydl.extract_info(link, download=True)

    
    keys = ['uploader','uploader_url','upload_date','creator','title','description','categories',
            'duration','view_count', 'like_count', 'dislike_count','average_rating','start_time', 'end_time',
            'release_date', 'release_year']

    filtered_d = dict((k, meta[k]) for k in keys if k in meta)
    df = pd.DataFrame.from_dict(filtered_d, orient='index').T
    df.index = df['title'] 
    files = os.listdir(source)

    for f in files: 
        if (f.startswith(str(meta['title']))):
            shutil.move(f, dest)
            #print('done')
        else: 
            #print(str(df['title']))
            continue

    
    sub_titles = glob.glob('./down/*.en.vtt')
    
    if len(sub_titles) != 0:
        vtt = webvtt.read(sub_titles[0])

        start_list = list()
        end_list = list()
        # Storing all the lines as part of the lines list
        lines = []


        for x in range(len(vtt)):
            start_list.append(vtt[x].start)
            end_list.append(vtt[x].end)


        for line in vtt:
            lines.append(line.text.strip().splitlines())

            lines = [' '.join(item) for item in lines]
            
        final_df = pd.DataFrame({'Start_time': start_list, 'End_time': end_list, 'Statement': lines})

        sid_obj = SentimentIntensityAnalyzer()
        sentiment_scores_vader = [sid_obj.polarity_scores(article) for article in final_df.Statement]

        sentiment_category_positive = []
        sentiment_category_neutral = []
        sentiment_category_negative = []
        sentiment_category_compound = []

        for sentiments in sentiment_scores_vader:
            sentiment_category_positive.append(sentiments['pos'])
            sentiment_category_neutral.append(sentiments['neu'])
            sentiment_category_negative.append(sentiments['neg'])
            sentiment_category_compound.append(sentiments['compound'])
        
        sentiment_df = pd.DataFrame([[article for article in final_df.Statement],
                                         sentiment_category_positive,
                                         sentiment_category_neutral,
                                         sentiment_category_negative,
                                         sentiment_category_compound]).T
        sentiment_df['Start_time'] = start_list
        sentiment_df['End_time'] = end_list
        sentiment_df.columns = ['Statement', 'positive_polarity', 'neutral_polarity', 'negative_polarity',
                                    'overall_polarity', 'Start_time', 'End_time']
        abcd = sentiment_df.to_json()
        
        return abcd
    return '''<form method ="POST">
                Enter the Youtube link <input type ="text" name ="link">
                <input type ="submit">
                </form>'''

#app.run(debug=True)

In [ ]:
link =  "https://www.youtube.com/watch?v=LdfSJabF2NI"
scraping_download(link)

[youtube] LdfSJabF2NI: Downloading webpage


[youtube] LdfSJabF2NI: Looking for automatic captions
[info] Writing video subtitles to: China censors research on CCP virus origin; Wuhan volunteer threatened after speaking about outbreak-LdfSJabF2NI.en.vtt
[info] Writing video description metadata as JSON to: China censors research on CCP virus origin; Wuhan volunteer threatened after speaking about outbreak-LdfSJabF2NI.info.json
[download] Destination: China censors research on CCP virus origin; Wuhan volunteer threatened after speaking about outbreak-LdfSJabF2NI.webm
[download] 100% of 20.85MiB in 00:0272MiB/s ETA 00:00known ETA
[ffmpeg] Destination: China censors research on CCP virus origin; Wuhan volunteer threatened after speaking about outbreak-LdfSJabF2NI.wav
Deleting original file China censors research on CCP virus origin; Wuhan volunteer threatened after speaking about outbreak-LdfSJabF2NI.webm (pass -k to keep)


In [ ]:
import webvtt

vtt = webvtt.read("down/zbjmnwfc.en.vtt")
start_list = list()
end_list = list()
# Storing all the lines as part of the lines list
lines = []


for x in range(len(vtt)):
    start_list.append(vtt[x].start)
    end_list.append(vtt[x].end)


for line in vtt:
    lines.append(line.text.strip().splitlines())

    lines = [' '.join(item) for item in lines]

final_df = pd.DataFrame({'Start_time': start_list, 'End_time': end_list, 'Statement': lines})

In [ ]:
final_df

In [ ]:
files

In [53]:
#f.endswith('.wav')#('We explain the COVID curve, herd immunity, R-naught & pool-testing-4zF7IQ_xqTQ')

In [56]:
for f in files: 
    if (f.startswith('We explain the COVID curve, herd immunity, R-naught & pool')):
        shutil.move(f, dest)

In [71]:
sub_titles = glob.glob('./*.en.vtt')

if len(sub_titles) != 0:
    print(f)
else:    
    print('Automatic caption is not be created by YouTube.')

Automatic caption is not be created by YouTube.


In [82]:
#files.starts_with("We explain the COVID curve, herd immunity, R-naught & pool")

In [83]:
files

['We explain the COVID curve, herd immunity, R-naught & pool-testing-4zF7IQ_xqTQ.info.json',
 'We explain the COVID curve, herd immunity, R-naught & pool-testing-4zF7IQ_xqTQ.wav']

In [84]:
source = os.getcwd()
dest = os.getcwd() + '/down/'
files = os.listdir(source)

for f in files: 
    if (f.startswith('We explain the COVID curve, herd immunity, R-naught & pool')): # change it to appropriate
        print('True')

True
True


In [87]:
#sub_titles = glob.glob('./down/*.en.vtt')
#sub_titles = 
glob.glob('./down/*.en.vtt')

["./down/You're wrong about more than you think.-ioxWuCd-mn0.en.vtt"]